# Mock community dataset generation

Run in a [qiime 2.0.6 conda environment](https://docs.qiime2.org/2.0.6/install/).

This notebook describes how mock community datasets were retrieved and files were generated for tax-credit comparisons. Only the feature tables, metadata maps, representative sequences, and expected taxonomies are included in tax-credit, but this notebook can regenerate intermediate files, generate these files for new mock communities, or tweaked to benchmark, e.g., quality control or OTU picking methods.

All mock communities are hosted on [mockrobiota](http://caporasolab.us/mockrobiota/), though raw reads are deposited elsewhere. To use these mock communities, clone the ``mockrobiota`` repository into the ``repo_dir`` that contains the tax-credit repository.

In [1]:
from tax_credit.process_mocks import *
from os.path import expandvars, join

Set source/destination filepaths

In [2]:
# base directory containing tax-credit and mockrobiota repositories
repo_dir = expandvars("$HOME/Desktop/projects/")
# tax-credit directory
project_dir = join(repo_dir, "short-read-tax-assignment")
# mockrobiota directory
mockrobiota_dir = join(repo_dir, "mockrobiota")
# temp destination for mock community files
mock_data_dir = join(repo_dir, "mock-community")
# destination for expected taxonomy assignments
expected_data_dir = join(project_dir, "data", "precomputed-results", "mock-community")


First we will define which mock communities we plan to use, and necessary parameters

In [3]:
# We will just use a sequential set of mockrobiota datasets, otherwise list community names manually
communities = ['mock-{0}'.format(n) for n in range(1,11)]

# Create dictionary of mock community dataset metadata
community_metadata = extract_mockrobiota_dataset_metadata(mockrobiota_dir, communities)

# Map marker-gene to reference database names in tax-credit and in mockrobiota
#           marker-gene  tax-credit-dir  mockrobiota-dir version
reference_dbs = {'16S' : ('gg_13_8_otus', 'greengenes', '13_8'),
                 'ITS' : ('unite_20.11.2016', 'unite', '97')
                }

Now we will generate data directories in ``tax-credit`` for each community and begin populating these will files from ``mockrobiota``. This may take some time, as this involves downloading raw data fastq files.

In [4]:
extract_mockrobiota_data(communities, community_metadata, reference_dbs, 
                         mockrobiota_dir, mock_data_dir, 
                         expected_data_dir)

## Process data in QIIME2
Finally, we can get to processing our data. We begin by importing our data, demultiplexing, and viewing a few fastq quality summaries to decide how to trim our raw reads prior to processing.

Each dataset may require different parameters. For example, some mock communities used here require different barcode orientations. These parameters may be read in as a dictionary of tuples.

In [5]:
# {community : (rev_comp_barcodes, rev_comp_mapping_barcodes)}
demux_params = {'mock-1' : (False, True),
               'mock-2' : (False, True),
               'mock-3' : (False, False),
               'mock-4' : (False, True),
               'mock-5' : (False, True),
               'mock-6' : (False, True),
               'mock-7' : (False, True),
               'mock-8' : (False, True),
               'mock-9' : (False, True),
               'mock-10' : (False, True)
              }

In [6]:
batch_demux(communities, mock_data_dir, demux_params)

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: profile_quality.R /var/folders/0t/l1dz88p50y91vx_22fx5w3780000gn/T/qiime2-archive-46vxmj3v/8ae9f3dc-8e4f-47e2-834b-75bbfac317f1/data/HMPMockV1.2.Staggered2_4_L001_R1_001.fastq.gz /var/folders/0t/l1dz88p50y91vx_22fx5w3780000gn/T/qiime2-temp-y9urhpgi

mock-3 complete


To view the ``demux_summary.qzv`` (demultiplexed sequences per sample counts) and ``demux_plot_qual.qzv`` (fastq quality profiles) summaries that you just created, drag and drop the files into [q2view](https://view.qiime2.org/)

Use the fastq quality data above to decide how to proceed. As each dataset will have different quality profiles and read lengths, we will enter trimming parameters as a dictionary. We can use this dict to pass other parameters to ``denoise_to_phylogeny()``, including whether we want to build a phylogeny for each community.

In [6]:
# {community : (trim_left, trunc_len, build_phylogeny)}
trim_params = {'mock-1' : (0, 100, True),
               'mock-2' : (0, 130, True),
               'mock-3' : (0, 150, True),
               'mock-4' : (0, 150, True),
               'mock-5' : (0, 200, True),
               'mock-6' : (0, 50, True),
               'mock-7' : (0, 90, True),
               'mock-8' : (0, 100, True),
               'mock-9' : (0, 100, False),
               'mock-10' : (0, 100, False)
              }

Now we will quality filter with ``dada2``, and use the representative sequences to generate a phylogeny.

In [28]:
denoise_to_phylogeny(communities, mock_data_dir, trim_params)

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada.R /var/folders/0t/l1dz88p50y91vx_22fx5w3780000gn/T/qiime2-archive-yy2wiw0u/d6db247c-3690-41fb-b56c-07120727b815/data /var/folders/0t/l1dz88p50y91vx_22fx5w3780000gn/T/tmpc5ewyppv/output.tsv.biom 150 0 2 2 /var/folders/0t/l1dz88p50y91vx_22fx5w3780000gn/T/tmpc5ewyppv

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: mafft --preservecase /var/folders/0t/l1dz88p50y91vx_22fx5w3780000gn/T/qiime2-archive-jzdwb10g/a8035a93-2717-42c7-9b65-2724958499cd/data/dna-sequences.fasta

Running external command line application. This may print messages to stdout and/or stderr.
T

In [32]:
for num in range(1,11):
    seq_fp = '/Users/nbokulich/Desktop/projects/short-read-tax-assignment/data/mock-community/mock-{0}/rep_seqs.qza'.format(num)
    test_out = '/Users/nbokulich/Desktop/projects/short-read-tax-assignment/data/mock-community/mock-{0}/rep_seqs.fna'.format(num)
    a = Artifact.load(seq_fp)
    io.write(a.view(DNAIterator).generator, format='fasta', into=test_out)

In [21]:
from q2_types import DNAIterator
seqs = a.view(DNAIterator)
#len(seqs)

In [30]:
from skbio import io


'/Users/nbokulich/Desktop/projects/short-read-tax-assignment/data/mock-community/mock-3/test.txt'

To view the ``feature_table_summary.qzv`` summaries you just created, drag and drop the files into [q2view](https://view.qiime2.org/)

## Extract results and move to repo

In [5]:
transport_to_repo(communities, mock_data_dir, project_dir)